<style>
    body, * {
        direction: rtl !important;
        text-align: right !important;
    }
</style>
מקור: [Spark By Examples - PySpark Broadcast Variables](https://sparkbyexamples.com/pyspark/pyspark-broadcast-variables/)


# משתני Broadcast ב-PySpark

## מבוא

ב-PySpark RDD ו-DataFrame, משתני Broadcast הם **משתנים לקריאה בלבד** שנשמרים במטמון וזמינים בכל הצמתים באשכול (cluster). במקום לשלוח את הנתונים הללו עם כל משימה (task), PySpark מפיץ את משתני ה-broadcast לעובדים (workers) באמצעות אלגוריתמים יעילים כדי להפחית עלויות תקשורת.

### מתי להשתמש במשתני Broadcast?

נניח שאתם מקבלים קובץ עם קודי מדינות בעלי שתי אותיות ואתם רוצים להמיר אותו לשמות מדינות מלאים (לדוגמה: CA ל-California, NY ל-New York וכו') באמצעות lookup או מיפוי התייחסות. במקרים מסוימים, הנתונים הללו יכולים להיות גדולים ויתכן שיהיו לכם חיפושים רבים (כמו zipcode וכו').

במקום להפיץ את המידע הזה יחד עם כל משימה ברחבי הרשת (overhead וזמן רב), אנחנו יכולים להשתמש במשתני broadcast כדי לשמור את מידע החיפוש הזה במטמון בכל מכונה ולהשתמש במידע השמור במטמון בזמן ביצוע הטרנספורמציות.

---

## איך עובד Broadcast ב-PySpark?

משתני Broadcast משמשים באותה דרך עבור RDD ו-DataFrame.

כאשר מריצים יישום PySpark RDD או DataFrame שבו מוגדרים ונעשה שימוש במשתני Broadcast, PySpark מבצע את הפעולות הבאות:

1. **PySpark מפרק את העבודה לשלבים** (stages) שמכילים משימות מופצות ופעולות שמבוצעות בשלב
2. **שלבים מאוחרים יותר גם הם מחולקים למשימות**
3. **Spark משדר את הנתונים המשותפים (הניתנים לשימוש חוזר) הנדרשים על ידי משימות בכל שלב**
4. **הנתונים המשודרים נשמרים במטמון בפורמט סדרתי ומבוטל-סדרתי לפני ביצוע כל משימה**

**חשוב:** אתם צריכים ליצור ולהשתמש במשתני broadcast עבור נתונים שמשותפים בין מספר שלבים ומשימות.

**שימו לב:** משתני broadcast לא נשלחים למבצעים (executors) עם `sc.broadcast(variable)` - במקום זאת, הם יישלחו למבצעים כאשר נעשה בהם שימוש לראשונה.

---

## איך ליצור משתנה Broadcast

ה-Broadcast ב-PySpark נוצר באמצעות המתודה `broadcast(v)` של המחלקה SparkContext. מתודה זו מקבלת את הארגומנט `v` שאתם רוצים לשדר.

### ב-PySpark Shell

הסינטקס הבסיסי ליצירת משתנה broadcast הוא:

```python
broadcastVar = sc.broadcast(Array(1, 2, 3))
broadcastVar.value
```

---

## דוגמה למשתנה Broadcast ב-PySpark RDD

להלן דוגמה פשוטה מאוד לאופן השימוש במשתני broadcast ב-RDD. 

הדוגמה מגדירה נתונים נפוצים (states - מדינות) במשתנה Map ומפיצה את המשתנה באמצעות `SparkContext.broadcast()` ולאחר מכן משתמשת במשתנים אלה ב-RDD על טרנספורמציה של `map()`.

### ייבוא הספריות והתחלת Spark Session

In [ ]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

### הגדרת משתנה Broadcast עם נתוני המדינות

In [ ]:
states = {"NY":"New York", "CA":"California", "FL":"Florida"}
broadcastStates = spark.sparkContext.broadcast(states)

### יצירת נתוני דוגמה

In [ ]:
data = [("James","Smith","USA","CA"),
        ("Michael","Rose","USA","NY"),
        ("Robert","Williams","USA","CA"),
        ("Maria","Jones","USA","FL")
       ]

### יצירת RDD והמרה באמצעות משתנה ה-Broadcast

In [ ]:
rdd = spark.sparkContext.parallelize(data)

def state_convert(code):
    return broadcastStates.value[code]

result = rdd.map(lambda x: (x[0], x[1], x[2], state_convert(x[3]))).collect()
print(result)

**הפלט הצפוי:**

```
[('James', 'Smith', 'USA', 'California'), 
 ('Michael', 'Rose', 'USA', 'New York'), 
 ('Robert', 'Williams', 'USA', 'California'), 
 ('Maria', 'Jones', 'USA', 'Florida')]
```

---

## דוגמה למשתנה Broadcast ב-PySpark DataFrame

להלן דוגמה לאופן השימוש במשתני broadcast ב-DataFrame, בדומה לדוגמה הקודמת עם RDD.

גם כאן משתמשים בנתונים נפוצים (states) במשתנה Map ומפיצים את המשתנה באמצעות `SparkContext.broadcast()` ולאחר מכן משתמשים במשתנים אלה בטרנספורמציה של `map()` על DataFrame.

**הערה:** אם אינכם מכירים את DataFrame, מומלץ ללמוד על DataFrame לפני שממשיכים במאמר זה.

### ייבוא הספריות

In [ ]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

### הגדרת משתנה Broadcast

In [ ]:
states = {"NY":"New York", "CA":"California", "FL":"Florida"}
broadcastStates = spark.sparkContext.broadcast(states)

### יצירת נתוני דוגמה

In [ ]:
data = [("James","Smith","USA","CA"),
        ("Michael","Rose","USA","NY"),
        ("Robert","Williams","USA","CA"),
        ("Maria","Jones","USA","FL")
       ]

### יצירת DataFrame

In [ ]:
columns = ["firstname", "lastname", "country", "state"]
df = spark.createDataFrame(data=data, schema=columns)
df.printSchema()
df.show(truncate=False)

### המרת הנתונים באמצעות משתנה Broadcast

In [ ]:
def state_convert(code):
    return broadcastStates.value[code]

result = df.rdd.map(lambda x: (x[0], x[1], x[2], state_convert(x[3]))).toDF(columns)
result.show(truncate=False)

**הפלט הצפוי:**

```
+---------+--------+-------------+----------+
|firstname|lastname|country      |state     |
+---------+--------+-------------+----------+
|James    |Smith   |United States|California|
|Michael  |Rose    |United States|New York  |
|Robert   |Williams|United States|California|
|Maria    |Jones   |United States|Florida   |
+---------+--------+-------------+----------+
```

### שימוש במשתנה Broadcast ב-Filter ו-Join

ניתן גם להשתמש במשתנה broadcast על filter ו-joins. להלן דוגמה לשימוש ב-filter:

In [ ]:
# Broadcast variable on filter
filteDf = df.where(df['state'].isin(list(broadcastStates.value)))
filteDf.show()

---

## סיכום

במדריך זה על משתני Broadcast ב-PySpark, למדתם:
- מהו משתנה Broadcast
- מהם היתרונות שלו
- כיצד להשתמש בו ב-RDD וב-DataFrame עם דוגמאות מעשיות

---

## מקורות נוספים

- [PySpark SparkContext Explained](https://sparkbyexamples.com/pyspark/pyspark-sparkcontext/)
- [Dynamic way of doing ETL through PySpark](https://sparkbyexamples.com/pyspark/dynamic-way-of-doing-etl-through-pyspark/)
- [PySpark Shell Command Usage with Examples](https://sparkbyexamples.com/pyspark/pyspark-shell-command/)
- [PySpark Accumulator with Example](https://sparkbyexamples.com/pyspark/pyspark-accumulator-with-example/)
- [PySpark distinct vs dropDuplicates](https://sparkbyexamples.com/pyspark/pyspark-distinct-vs-dropduplicates/)
- [PySpark Get Number of Rows and Columns](https://sparkbyexamples.com/pyspark/pyspark-get-number-of-rows-and-columns/)
- [Pyspark Select Distinct Rows](https://sparkbyexamples.com/pyspark/pyspark-select-distinct-rows/)

### תיעוד רשמי

- [Apache Spark - Broadcast Variables Documentation](https://spark.apache.org/docs/2.2.0/rdd-programming-guide.html#broadcast-variables)

---

**למידה מהנה! 🎉**